# 1. Loading and processing the data

1.1. Cargamos la data

In [48]:
# Importamos paquetes relevantes
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

In [2]:
#Definimos el dataframe
df = pd.read_csv('../../../../data/wage2015_subsample_inference.csv')
display(df)

,rownames,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,...,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
0,10,9.615385,2.263364,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
1,12,48.076923,3.872802,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
2,15,11.057692,2.403126,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
3,18,13.942308,2.634928,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
4,19,28.846154,3.361977,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5145,32620,14.769231,2.692546,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,9.0,0.81,0.729,0.6561,4700.0,16,4970.0,9
5146,32624,23.076923,3.138833,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,12.0,1.44,1.728,2.0736,4110.0,13,8680.0,20
5147,32626,38.461538,3.649659,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,11.0,1.21,1.331,1.4641,1550.0,4,3680.0,6
5148,32631,32.967033,3.495508,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,10.0,1.00,1.000,1.0000,2920.0,9,6570.0,11


1.2. Generamos el modelo extra flexible

In [3]:
LogSalario = pd.DataFrame(df['lwage'])
Variables = pd.DataFrame(df.drop(columns=['rownames', 'lwage', 'wage']))

1.2.1. Generar el array para Y y normalizarlo

In [13]:
Y = LogSalario.to_numpy().reshape(-1, 1)

Media_Y = np.mean(Y)
Desv_Std_Y = np.std(Y)

Y_normal = (Y-Media_Y) / Desv_Std_Y

Y_normal

array([[-1.24037498],
       [ 1.5815695 ],
       [-0.99532021],
       ...,
       [ 1.19031569],
       [ 0.9200321 ],
       [-0.20976589]])

1.2.2. Generar el array para las variables X y normalizarlo

In [5]:
Variables_base = ["sex", "hsg", "scl", "clg", "ad", "so", "we", "ne", "exp1", "occ2", "ind2"]
X_base = pd.get_dummies(df[Variables_base], columns = ["occ2", "ind2"], drop_first = True, dtype = int)

X_extra_flexible = pd.concat((X_base, df[["exp2", "exp3", "exp4"]]), axis=1)
X_extra_flexible_features = X_extra_flexible.columns

In [6]:
base_experience_variables = ["exp1", "exp2", "exp3"]
exp_4 = ["exp4"]
education_variables = ["hsg", "scl", "clg", "ad"]
location_variables = ["so", "we", "ne"]
occupation_variables = [variable for variable in X_extra_flexible.columns if "occ2" in variable]
industry_variables = [variable for variable in X_extra_flexible.columns if "ind2" in variable]
variable_groups = [base_experience_variables, exp_4, education_variables, location_variables, occupation_variables, industry_variables]

In [7]:
for variable_group_index, variable_group in enumerate(variable_groups, 1):
        if variable_group == len(variable_groups):
                break

        other_variable_groups = variable_groups[variable_group_index:]

        for variable in variable_group:
                for other_variables in other_variable_groups:
                        temp = X_extra_flexible[other_variables].multiply(X_extra_flexible[variable], axis = "index")
                        temp = temp.add_prefix(f"{variable}")
                        X_extra_flexible = pd.concat(
                                (X_extra_flexible, temp),
                                axis = 1
                        )

In [14]:
temp = pd.DataFrame({"exp8": X_extra_flexible["exp4"] ** 2})
Modelo_extra_flexible = pd.concat((X_extra_flexible, temp), axis = 1)

In [28]:
Media_X = Modelo_extra_flexible.mean(axis=0)
Desv_Std_X = Modelo_extra_flexible.std(axis=0)

Modelo_X_flex_normal = (Modelo_extra_flexible-Media_X) / Desv_Std_X

Modelo_X_flex_normal

,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ2_2,...,occ2_22ind2_14,occ2_22ind2_15,occ2_22ind2_16,occ2_22ind2_17,occ2_22ind2_18,occ2_22ind2_19,occ2_22ind2_20,occ2_22ind2_21,occ2_22ind2_22,exp8
0,1.117877,-0.567878,-0.620547,1.465437,-0.398541,-0.649147,-0.525020,1.841139,-0.637222,-0.262056,...,-0.013935,NaN,-0.057544,-0.02414,-0.02414,-0.013935,-0.034149,-0.031171,-0.027877,-0.282275
1,-0.894379,-0.567878,-0.620547,1.465437,-0.398541,-0.649147,-0.525020,1.841139,1.624909,-0.262056,...,-0.013935,NaN,-0.057544,-0.02414,-0.02414,-0.013935,-0.034149,-0.031171,-0.027877,0.406406
2,-0.894379,1.760601,-0.620547,-0.682258,-0.398541,-0.649147,-0.525020,1.841139,0.399588,-0.262056,...,-0.013935,NaN,-0.057544,-0.02414,-0.02414,-0.013935,-0.034149,-0.031171,-0.027877,-0.273381
3,1.117877,-0.567878,-0.620547,-0.682258,2.508664,-0.649147,-0.525020,1.841139,1.059376,-0.262056,...,-0.013935,NaN,-0.057544,-0.02414,-0.02414,-0.013935,-0.034149,-0.031171,-0.027877,-0.159069
4,1.117877,-0.567878,-0.620547,1.465437,-0.398541,-0.649147,-0.525020,1.841139,0.776610,-0.262056,...,-0.013935,NaN,-0.057544,-0.02414,-0.02414,-0.013935,-0.034149,-0.031171,-0.027877,-0.237969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5145,-0.894379,-0.567878,-0.620547,1.465437,-0.398541,-0.649147,1.904319,-0.543037,-0.448711,-0.262056,...,-0.013935,NaN,-0.057544,-0.02414,-0.02414,-0.013935,-0.034149,-0.031171,-0.027877,-0.282245
5146,1.117877,-0.567878,1.611169,-0.682258,-0.398541,-0.649147,1.904319,-0.543037,-0.165945,-0.262056,...,-0.013935,NaN,-0.057544,-0.02414,-0.02414,-0.013935,-0.034149,-0.031171,-0.027877,-0.281932
5147,-0.894379,-0.567878,-0.620547,-0.682258,2.508664,-0.649147,1.904319,-0.543037,-0.260200,-0.262056,...,-0.013935,NaN,-0.057544,-0.02414,-0.02414,-0.013935,-0.034149,-0.031171,-0.027877,-0.282107
5148,-0.894379,1.760601,-0.620547,-0.682258,-0.398541,-0.649147,1.904319,-0.543037,-0.354455,-0.262056,...,-0.013935,NaN,-0.057544,-0.02414,-0.02414,-0.013935,-0.034149,-0.031171,-0.027877,-0.282199


1.2.3. Separar las muestras de test y training al 10%

In [34]:
X_extra_flexible_train, X_extra_flexible_test, Y_extra_flexible_train, Y_extra_flexible_test = train_test_split(X_extra_flexible, Y, train_size = .1, shuffle = True)

# 2. Crear el procedimiento Lasso Cross-Validation

2.4. Definir una fucnión que cree una cuadrícula logarítmica

In [44]:
def log_spaced_grid(start, stop, num):
    return np.logspace(start, stop, num)

2.5. Programar una función que genere un número k de folds

In [57]:
def generate_folds(X, Y, n_splits):
    
    # Crear el grupo de índices
    indices = np.arange(len(X))
    
    # Determinar el tamaño de los folds
    fold_sizes = len(X) // n_splits
    remainder = len(X) % n_splits
    
    folds = []
    current = 0

    for i in range(n_splits):

        # Establecer el index final de cada fold
        end = current + fold_sizes + (1 if i < remainder else 0)
        
        # Definir los index para test y training
        test_indices = indices[current:end]
        train_indices = np.concatenate((indices[:current], indices[end:]))
        
        folds.append((train_indices, test_indices))
        
        # Actualizar los index
        current = end

    return folds

2.6. Con los items anteriores, programar una función que halle un valor λ tal que minimize el error de la media del error cuadrado en los folds

In [61]:
#Definimos la función Lasso

def lasso_cv(Y, X, lambda_bounds, k):
    
    lower_lambda, upper_lambda = lambda_bounds
    all_lambdas = np.logspace(np.log10(lower_lambda), np.log10(upper_lambda), num=100)
    
    all_mse = []

    for lambda_val in all_lambdas:
        mse_folds = []

        for train_idx, test_idx in folds:
            lasso_model = Lasso(alpha=lambda_val)
            lasso_model.fit(X[train_idx], Y[train_idx])
            Y_pred = lasso_model.predict(X[test_idx])
            mse = mean_squared_error(Y[test_idx], Y_pred)
            mse_folds.append(mse)
        
        all_mse.append(np.mean(mse_folds))

    optimal_idx = np.argmin(all_mse)
    optimal_lambda = all_lambdas[optimal_idx]
    
    # Definir los resultados de λ en base al entrenamiento del modelo
    optimal_model = Lasso(alpha=optimal_lambda)
    optimal_model.fit(X, Y)
    optimal_coef = optimal_model.coef_

    return {
        'optimal_lambda': optimal_lambda,
        'optimal_coef': optimal_coef,
        'all_lambdas': all_lambdas,
        'all_mse': all_mse
    }

2.7. Programar una función para predecir las variables de los resultados del modelo con el λ óptimo

In [60]:
def predict_with_optimal_lambda(optimal_model, X):
    
    # Extraer λ óptimo y los coeficientes del modelo
    optimal_lambda = optimal_model['optimal_lambda']
    optimal_coef = optimal_model['optimal_coef']

    # Definir el modelo de acuerdo a la función Lasso con el λ óptimo
    lasso_model = Lasso(alpha=optimal_lambda)
    lasso_model.coef_ = optimal_coef

    # Realizar la predicción con los predictores planteados
    predictions = lasso_model.predict(X)

    return predictions

# 3. Aplicar el procedimiento Lasso Cross-Validation

3.8. Aplicar un modelo OSL simple con la muestra de training

3.9. Encuentra el lambda óptimo y los coeficientes de la función programada en el item 3. y la muestra de training.  

El logaritmo natural de los números en la cuadrícula de λs deben estar espaciados por 0.1
El logaritmo natural del los límetos inferior y superior son -7 y +7, respectivamente.
Emplear 5 folds